# XGBoost Model

In [23]:
# Load the necessary packages
from darts import TimeSeries
from darts.models import XGBModel
import pandas as pd
import plotly.graph_objs as go
import numpy as np


In [24]:
# Load in the train and test data
train_df = pd.read_csv('../../data/Final_data/train_df.csv')
test_df = pd.read_csv('../../data/Final_data/test_df.csv')

train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

# Create the time series
series_train = TimeSeries.from_dataframe(train_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')
series_test = TimeSeries.from_dataframe(test_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')

# Define the future covariates columns from your dataframe
future_covariates_columns = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 
                             'Biomass (GWh)', 'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 
                             'Natural_gas (GWh)', 'Other (GWh)', 'Pumped_storage_generation (GWh)', 
                             'Solar_energy (GWh)', 'Wind_offshore (GWh)', 'Wind_onshore (GWh)', 
                             'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)', 
                             'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)']

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_test = TimeSeries.from_dataframe(test_df, 'Date', future_covariates_columns).astype('float32')

# Concatenate the train and test data
df = pd.concat([train_df, test_df])
df['Date'] = pd.to_datetime(df['Date'])

df

,Date,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
0,2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.10,250.979
1,2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.00,258.671
2,2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.90,271.495
3,2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.40,270.613
4,2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.30,287.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,2024-07-24,66.61,225.04,3.47,17.54,110.007,43.469,85.857,199.246,194.291,54.026,20.934,325.285,49.360,179.921,-168.705,992,75.75,32.63,0.000
755,2024-07-25,78.34,272.71,2.12,17.85,110.410,50.676,82.632,195.983,209.610,52.963,18.766,394.116,51.053,42.885,-194.496,992,76.36,31.70,0.000
756,2024-07-26,93.04,172.33,2.60,19.09,110.852,42.333,79.531,205.273,205.773,52.616,19.081,256.246,40.449,129.267,-241.786,993,75.21,32.20,0.000
757,2024-07-27,80.74,176.67,2.05,19.63,110.479,33.307,74.958,184.012,216.412,50.927,18.856,244.051,2.180,32.001,-251.655,992,74.79,32.90,0.000


### Hyperparameter Tuning

##### Here only using the split into train and test set

In [25]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import XGBModel
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MaxAbsScaler, RobustScaler, StandardScaler
import plotly.graph_objs as go
from darts.metrics import mape, rmse, mse, mae
import optuna

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Determine required start date for future covariates
input_chunk_length = 500  # Set based on desired look-back period
required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=input_chunk_length)

# Ensure future_covariates_full covers the required range
required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test)-1)

# Check if future_covariates_full has sufficient data
if future_covariates_full.start_time() > required_start_date or future_covariates_full.end_time() < required_end_date:
    print("Warning: The future_covariates_full is not long enough to cover the required input chunk length and prediction range.")
    # Extend the future_covariates_full or adjust your dataset

# Slice the future covariates to the required range, including data from the training period
future_covariates_test = future_covariates_full.slice(required_start_date, required_end_date)

# Scaling the data
scaler_series = Scaler(MaxAbsScaler())
scaler_covariates = Scaler(MaxAbsScaler())

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series and future covariates using the same scaler
series_test_scaled = scaler_series.transform(series_test)
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

# Define the Optuna objective function
def objective(trial):
    # Define hyperparameters to tune
    max_depth = trial.suggest_int('max_depth', 3, 15)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.3, log=True)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    input_chunk_length = trial.suggest_int('input_chunk_length', 10, 300)
    min_child_weight = trial.suggest_float('min_child_weight', 0.1, 10.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 5)
    
    model = XGBModel(
        lags=input_chunk_length,
        output_chunk_length=1,
        lags_future_covariates=[0],
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        random_state=42
    )

    # Train the model
    model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, verbose=False) 

    # Make predictions on the test set
    forecast_scaled = model.predict(n=len(series_test_scaled), future_covariates=future_covariates_test_scaled)

    # Inverse transform the forecast to the original scale
    forecast = scaler_series.inverse_transform(forecast_scaled)

    # Evaluate the model performance using MAPE
    error = mse(series_test, forecast)

    return error

# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500, n_jobs=7)

# Print the best hyperparameters
print('/n Best hyperparameters: ', study.best_params)

# Extract the best hyperparameters from the Optuna study
best_params = study.best_params

# Use the best hyperparameters to define the XGBoost model
best_model = XGBModel(
    lags=best_params['input_chunk_length'],
    output_chunk_length=1,  # Predicting one day at a time
    lags_future_covariates=[0],  # Using the current value of future covariates for prediction
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    min_child_weight=best_params['min_child_weight'],  
    subsample=best_params['subsample'],               
    colsample_bytree=best_params['colsample_bytree'], 
    gamma=best_params['gamma'],                       
    random_state=42
)

# Train the model with the best hyperparameters
best_model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled)

# Make predictions on the test set
forecast_scaled = best_model.predict(n=len(series_test_scaled), future_covariates=future_covariates_test_scaled)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# Convert TimeSeries to DataFrame for Plotly plotting
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

# Plot the results using Plotly
fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='XGBoost Forecast on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='XGBoost Model - Test Performance Only',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly_white'
)

# Show the plot
fig.show()


# Evaluate the model using Darts' metrics
print(f'Mean Absolute Error on Test Set: {mae(series_test, forecast)}')
print(f'Mean Absolute Percentage Error on Test Set: {mape(series_test, forecast)}')
print(f'Mean Squared Error on Test Set: {mse(series_test, forecast)}')
print(f'Root Mean Squared Error on Test Set: {rmse(series_test, forecast)}')



[I 2024-09-11 16:32:14,554] A new study created in memory with name: no-name-7bf12ae9-e63d-4069-a80e-a7886a97088d
[I 2024-09-11 16:32:17,853] Trial 2 finished with value: 13425.576171875 and parameters: {'max_depth': 10, 'learning_rate': 0.11412292578151693, 'n_estimators': 94, 'input_chunk_length': 174, 'min_child_weight': 6.684650061736955, 'subsample': 0.5995434823216879, 'colsample_bytree': 0.9530333606533643, 'gamma': 3.2094585686884747}. Best is trial 2 with value: 13425.576171875.
[I 2024-09-11 16:32:18,074] Trial 1 finished with value: 5174.0810546875 and parameters: {'max_depth': 14, 'learning_rate': 0.04592239719235583, 'n_estimators': 53, 'input_chunk_length': 267, 'min_child_weight': 4.084409377339601, 'subsample': 0.5285195582991558, 'colsample_bytree': 0.6142118508152623, 'gamma': 0.19988472135412827}. Best is trial 1 with value: 5174.0810546875.
[I 2024-09-11 16:32:18,109] Trial 0 finished with value: 12420.2451171875 and parameters: {'max_depth': 7, 'learning_rate': 0.0

/n Best hyperparameters:  {'max_depth': 14, 'learning_rate': 0.18973066180608145, 'n_estimators': 267, 'input_chunk_length': 285, 'min_child_weight': 1.6607694655354948, 'subsample': 0.846461608657852, 'colsample_bytree': 0.5605099283678379, 'gamma': 0.000967675267927029}


Mean Absolute Error on Test Set: 28.56793785095215
Mean Absolute Percentage Error on Test Set: 75.23526000976562
Mean Squared Error on Test Set: 2570.861572265625
Root Mean Squared Error on Test Set: 50.70366287231445


In [27]:
# Save the created figure as png file and the error metrics 
fig.write_image("../../predictions/XGBoost/XGBoost_forecast_no_tscv.png")
error_metrics = pd.DataFrame({'MAE': [mae(series_test, forecast)], 'MAPE': [mape(series_test, forecast)], 'MSE': [mse(series_test, forecast)], 'RMSE': [rmse(series_test, forecast)]})
error_metrics.to_csv('../../predictions/XGBoost/XGBoost_error_metrics_no_tscv.csv', index=False)

# Also save hyperparameters of the model in the csv file
best_params_df = pd.DataFrame(best_params, index=[0])
best_params_df.to_csv('../../predictions/XGBoost/XGBoost_hyperparameters_no_tscv.csv', index=False)

##### Applying the TimeSeriesSplit

In [30]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import XGBModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mse, mae
import optuna
from sklearn.model_selection import TimeSeriesSplit
import plotly.graph_objs as go

# Function to train the model and evaluate performance using cross-validation
def train_and_evaluate(tscv, series_train_scaled, future_covariates_train_scaled, scaler_series, n_splits):
    def objective(trial):
        # Define hyperparameters to tune
        max_depth = trial.suggest_int('max_depth', 3, 15)
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.3, log=True)
        n_estimators = trial.suggest_int('n_estimators', 50, 500)
        input_chunk_length = trial.suggest_int('input_chunk_length', 10, 200)
        min_child_weight = trial.suggest_float('min_child_weight', 0.1, 10.0)
        subsample = trial.suggest_float('subsample', 0.5, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
        gamma = trial.suggest_float('gamma', 0, 5)
        output_chunk_length = trial.suggest_int('output_chunk_length', 1, 7)  # Tune for longer horizons

        # Initialize the model with the trial hyperparameters
        model = XGBModel(
            lags=input_chunk_length,
            output_chunk_length=output_chunk_length,
            lags_future_covariates=[0],
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            min_child_weight=min_child_weight,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            gamma=gamma,
            random_state=42
        )

        # Cross-validation loop
        errors_mse, errors_mae, errors_mape, errors_rmse = [], [], [], []

        for train_index, test_index in tscv.split(series_train_scaled.pd_dataframe()):
            # Check if there is enough data in the current fold to accommodate input_chunk_length
            if len(train_index) <= input_chunk_length:
                continue  # Skip this fold if not enough data

            # Get the corresponding date ranges from indices
            train_start, train_end = series_train_scaled.time_index[train_index[0]], series_train_scaled.time_index[train_index[-1]]
            test_start, test_end = series_train_scaled.time_index[test_index[0]], series_train_scaled.time_index[test_index[-1]]

            # Slice the TimeSeries objects based on date ranges
            train_series = series_train_scaled.slice(train_start, train_end)
            val_series = series_train_scaled.slice(test_start, test_end)
            train_covariates = future_covariates_train_scaled.slice(train_start, train_end)
            val_covariates = future_covariates_train_scaled.slice(test_start, test_end)

            # Train the model on the training set
            model.fit(train_series, future_covariates=train_covariates)

            # Predict on the validation set
            forecast_scaled = model.predict(n=len(val_series), future_covariates=val_covariates)

            # Inverse transform the forecast to the original scale
            forecast = scaler_series.inverse_transform(forecast_scaled)

            # Calculate errors on the validation set
            errors_mse.append(mse(val_series, forecast))
            errors_mae.append(mae(val_series, forecast))
            errors_rmse.append(rmse(val_series, forecast))

            # Only calculate MAPE if all actual values are positive
            if (val_series.values() > 0).all():
                errors_mape.append(mape(val_series, forecast))
            else:
                errors_mape.append(np.nan)  # or some default value to indicate MAPE could not be calculated

        # Store metrics for the current trial
        trial.set_user_attr('mse', np.mean(errors_mse))
        trial.set_user_attr('mae', np.mean(errors_mae))
        trial.set_user_attr('mape', np.nanmean(errors_mape))  # Use nanmean to handle NaN values
        trial.set_user_attr('rmse', np.mean(errors_rmse))

        # Return the average MSE across all splits as the metric to optimize
        return np.mean(errors_mse)

    # Create an Optuna study and optimize
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    study.optimize(objective, n_trials=300, n_jobs=5)

    return study

# Function to make final predictions and evaluate on the test set
def final_model_evaluation(best_params, series_train_scaled, future_covariates_train_scaled, series_test, future_covariates_test_scaled, scaler_series):
    best_model = XGBModel(
        lags=best_params['input_chunk_length'],
        output_chunk_length=best_params['output_chunk_length'], 
        lags_future_covariates=[0],  # Using the current value of future covariates for prediction
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        min_child_weight=best_params['min_child_weight'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        gamma=best_params['gamma'],
        random_state=42
    )

    # Fit the model on the entire scaled training set
    best_model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled)

    # Make predictions on the test set
    forecast_scaled = best_model.predict(n=len(series_test), future_covariates=future_covariates_test_scaled)

    # Inverse transform the forecast to the original scale
    forecast = scaler_series.inverse_transform(forecast_scaled)

    return forecast

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Scaling the data
scaler_series = Scaler()
scaler_covariates = Scaler()

# Fit the scaler on the full training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Define list of n_splits to try
n_splits_list = [5, 10]

# Initialize a dictionary to store DataFrames for each n_splits value
results_dict = {}

# Loop over different n_splits values
for n_splits in n_splits_list:
    print(f"Processing TimeSeriesSplit with {n_splits} splits...")

    # Define TimeSeriesSplit with the current n_splits
    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=500) 
    
    # Train and evaluate model using cross-validation
    study = train_and_evaluate(tscv, series_train_scaled, future_covariates_train_scaled, scaler_series, n_splits)

    # Print the best hyperparameters
    print(f'Best hyperparameters for {n_splits} splits: ', study.best_params)

    # Store the best hyperparameters for each split setting
    best_params = study.best_params

    # --- Slicing the Test Covariates According to Best `input_chunk_length` ---
    input_chunk_length = best_params['input_chunk_length']
    required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=input_chunk_length)
    required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test) - 1)
    future_covariates_test = future_covariates_full.slice(required_start_date, required_end_date)
    future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

    # Make final predictions on the test set using the best hyperparameters
    forecast = final_model_evaluation(best_params, series_train_scaled, future_covariates_train_scaled, series_test, future_covariates_test_scaled, scaler_series)

    # Store the results
    results_dict[f'n_splits_{n_splits}'] = pd.DataFrame({
        'mse': [mse(series_test, forecast)],
        'mae': [mae(series_test, forecast)],
        'mape': [mape(series_test, forecast) if (series_test.values() > 0).all() else np.nan],
        'rmse': [rmse(series_test, forecast)]
    })

# Find the best n_splits based on the lowest mse
comparison_df = pd.concat(
    [df.assign(n_splits=int(n_splits.split('_')[-1])) for n_splits, df in results_dict.items()]
)

best_n_splits = comparison_df['n_splits'].iloc[comparison_df['mse'].idxmin()]

# Corrected: Format the key properly to match the results_dict keys
best_n_splits_key = f'n_splits_{best_n_splits}'  # No need to convert to int, as it's already the correct format

# Extract the best hyperparameters for the best n_splits
best_results_df = results_dict[best_n_splits_key]
best_trial = best_results_df.loc[best_results_df['mse'].idxmin()]

print("\nBest hyperparameter combination and error metrics for the best TimeSeriesSplit configuration:")
print(f"Best n_splits: {best_n_splits_key}")
print(f"Best hyperparameters: {study.best_params}")
print(f"MSE: {best_trial['mse']}")
print(f"MAE: {best_trial['mae']}")
print(f"MAPE: {best_trial['mape']}")
print(f"RMSE: {best_trial['rmse']}")

# Evaluate error metrics on the full test data using the best hyperparameters
mse_test = mse(series_test, forecast)
mae_test = mae(series_test, forecast)
mape_test = mape(series_test, forecast) if (series_test.values() > 0).all() else np.nan
rmse_test = rmse(series_test, forecast)

print("\nError metrics for the test data using the best hyperparameters:")
print(f"MSE on Test Set: {mse_test}")
print(f"MAE on Test Set: {mae_test}")
print(f"MAPE on Test Set: {mape_test}")
print(f"RMSE on Test Set: {rmse_test}")

# Plot the results using Plotly for the best hyperparameters
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='XGBoost Forecast on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title=f'XGBoost Model - Test Performance (Best n_splits={best_n_splits_key})',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly'
)

# Show the plot
fig.show()


[I 2024-09-12 08:32:44,105] A new study created in memory with name: no-name-53cac99f-4efe-4735-a82a-4f17b4a97f33


Processing TimeSeriesSplit with 5 splits...


[I 2024-09-12 08:32:46,763] Trial 0 finished with value: 1305.2904052734375 and parameters: {'max_depth': 15, 'learning_rate': 0.10376625701919677, 'n_estimators': 134, 'input_chunk_length': 197, 'min_child_weight': 9.020844422618593, 'subsample': 0.8222321636466098, 'colsample_bytree': 0.735014683281811, 'gamma': 3.6552144118896974, 'output_chunk_length': 1}. Best is trial 0 with value: 1305.2904052734375.
[I 2024-09-12 08:32:48,940] Trial 3 finished with value: 1264.1754150390625 and parameters: {'max_depth': 10, 'learning_rate': 0.0013831997246393903, 'n_estimators': 63, 'input_chunk_length': 85, 'min_child_weight': 9.940053516671426, 'subsample': 0.7474545947552106, 'colsample_bytree': 0.9032172716029894, 'gamma': 3.8557296393496463, 'output_chunk_length': 6}. Best is trial 3 with value: 1264.1754150390625.
[I 2024-09-12 08:32:50,124] Trial 2 finished with value: 1260.43212890625 and parameters: {'max_depth': 13, 'learning_rate': 0.03472688149066481, 'n_estimators': 134, 'input_chu

Best hyperparameters for 5 splits:  {'max_depth': 9, 'learning_rate': 0.1608663590205178, 'n_estimators': 349, 'input_chunk_length': 15, 'min_child_weight': 9.167401721437601, 'subsample': 0.7804979728309989, 'colsample_bytree': 0.6941961501261891, 'gamma': 0.0035734059092539655, 'output_chunk_length': 1}


[I 2024-09-12 08:37:11,895] A new study created in memory with name: no-name-60fb6a2e-c99a-4ec5-8496-daeaf4ccc391


Processing TimeSeriesSplit with 10 splits...


[I 2024-09-12 08:37:16,732] Trial 2 finished with value: 1391.989501953125 and parameters: {'max_depth': 13, 'learning_rate': 0.058099840210773364, 'n_estimators': 255, 'input_chunk_length': 97, 'min_child_weight': 6.359926583498003, 'subsample': 0.9527965848684454, 'colsample_bytree': 0.9727169016323332, 'gamma': 1.9035559196919567, 'output_chunk_length': 1}. Best is trial 2 with value: 1391.989501953125.
[I 2024-09-12 08:37:17,168] Trial 1 finished with value: 1395.53076171875 and parameters: {'max_depth': 4, 'learning_rate': 0.1268007652031329, 'n_estimators': 87, 'input_chunk_length': 181, 'min_child_weight': 7.769866627515584, 'subsample': 0.5066029905490306, 'colsample_bytree': 0.8511211150237653, 'gamma': 2.527451455360621, 'output_chunk_length': 3}. Best is trial 2 with value: 1391.989501953125.
[I 2024-09-12 08:37:18,783] Trial 0 finished with value: 1395.72021484375 and parameters: {'max_depth': 15, 'learning_rate': 0.02014053134167667, 'n_estimators': 131, 'input_chunk_lengt

Best hyperparameters for 10 splits:  {'max_depth': 7, 'learning_rate': 0.24593457848977612, 'n_estimators': 328, 'input_chunk_length': 10, 'min_child_weight': 7.490516890310022, 'subsample': 0.7984300649642175, 'colsample_bytree': 0.8436499285402732, 'gamma': 4.29661959718613, 'output_chunk_length': 7}

Best hyperparameter combination and error metrics for the best TimeSeriesSplit configuration:
Best n_splits: n_splits_5
Best hyperparameters: {'max_depth': 7, 'learning_rate': 0.24593457848977612, 'n_estimators': 328, 'input_chunk_length': 10, 'min_child_weight': 7.490516890310022, 'subsample': 0.7984300649642175, 'colsample_bytree': 0.8436499285402732, 'gamma': 4.29661959718613, 'output_chunk_length': 7}
MSE: 4331.71484375
MAE: 43.07174301147461
MAPE: nan
RMSE: 65.81576538085938

Error metrics for the test data using the best hyperparameters:
MSE on Test Set: 13307.6220703125
MAE on Test Set: 73.95270538330078
MAPE on Test Set: nan
RMSE on Test Set: 115.3586654663086


#### Including self-written Cross-Validation

In [28]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import XGBModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mse, mae
import optuna
from optuna.pruners import MedianPruner
import plotly.graph_objs as go

# Custom cross-validation function
def custom_time_series_cv(series, n_splits, min_train_size):
    """ Custom cross-validation for time series that accounts for seasonality and other patterns. """
    series_length = len(series)
    fold_size = (series_length - min_train_size) // n_splits
    indices = np.arange(series_length)
    
    for i in range(n_splits):
        train_end = min_train_size + i * fold_size
        test_start = train_end
        test_end = test_start + fold_size

        if test_end > series_length:
            break  # Avoid out-of-bound indices
        
        yield indices[:train_end], indices[test_start:test_end]

# Function to train the model and evaluate performance using cross-validation with pruning
def train_and_evaluate(tscv_generator, series_train_scaled, future_covariates_train_scaled, scaler_series, n_splits):
    def objective(trial):
        # Define hyperparameters to tune
        max_depth = trial.suggest_int('max_depth', 3, 15)
        n_estimators = trial.suggest_int('n_estimators', 50, 500)
        input_chunk_length = trial.suggest_int('input_chunk_length', 50, 200) 
        min_child_weight = trial.suggest_float('min_child_weight', 0.1, 10.0)
        subsample = trial.suggest_float('subsample', 0.5, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
        # Refine hyperparameters with smaller ranges
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.05, log=True)
        gamma = trial.suggest_float('gamma', 0, 2)


        # Initialize the model with the trial hyperparameters (no model argument)
        model = XGBModel(
            lags=input_chunk_length,
            output_chunk_length=1,
            lags_future_covariates=[0],
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            min_child_weight=min_child_weight,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            gamma=gamma,
            random_state=42
        )

        # Cross-validation loop with early stopping
        errors_mse, errors_mae, errors_mape, errors_rmse = [], [], [], []

        # Initialize a counter for iterations
        i = 0

        # Cross-validation loop with early stopping
        for train_index, test_index in tscv_generator:
            # Check if there is enough data in the current fold to accommodate input_chunk_length
            if len(train_index) <= input_chunk_length:
                continue  # Skip this fold if not enough data

            # Get the corresponding date ranges from indices
            train_start, train_end = series_train_scaled.time_index[train_index[0]], series_train_scaled.time_index[train_index[-1]]
            test_start, test_end = series_train_scaled.time_index[test_index[0]], series_train_scaled.time_index[test_index[-1]]

            # Slice the TimeSeries objects based on date ranges
            train_series = series_train_scaled.slice(train_start, train_end)
            val_series = series_train_scaled.slice(test_start, test_end)
            train_covariates = future_covariates_train_scaled.slice(train_start, train_end)
            val_covariates = future_covariates_train_scaled.slice(test_start, test_end)

            # Train the model on the training set
            model.fit(train_series, future_covariates=train_covariates)

            # Predict on the validation set
            forecast_scaled = model.predict(n=len(val_series), future_covariates=val_covariates)

            # Inverse transform the forecast to the original scale
            forecast = scaler_series.inverse_transform(forecast_scaled)

            # Calculate errors on the validation set
            errors_mse.append(mse(val_series, forecast))
            errors_mae.append(mae(val_series, forecast))
            errors_rmse.append(rmse(val_series, forecast))

            # Only calculate MAPE if all actual values are positive
            if (val_series.values() > 0).all():
                errors_mape.append(mape(val_series, forecast))
            else:
                errors_mape.append(np.nan)  # Handle cases where MAPE cannot be calculated

            # Prune trial if not improving
            trial.report(np.mean(errors_mse), i)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

            # Increment the iteration counter
            i += 1

        # Safely calculate the average metrics
        mse_mean = np.nanmean(errors_mse) if errors_mse else np.inf
        mae_mean = np.nanmean(errors_mae) if errors_mae else np.inf
        rmse_mean = np.nanmean(errors_rmse) if errors_rmse else np.inf
        mape_mean = np.nanmean(errors_mape) if errors_mape else np.nan

        # Return the average MSE across all splits as the metric to optimize
        if np.isnan(mse_mean):
            return np.inf  # Return a high value if the result is NaN
        return mse_mean

    # Create an Optuna study with a pruner and optimize
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner()))
    study.optimize(objective, n_trials=100, n_jobs=5)

    return study

# Function to make final predictions and evaluate on the test set
def final_model_evaluation(best_params, series_train_scaled, future_covariates_train_scaled, series_test, future_covariates_test_scaled, scaler_series):
    best_model = XGBModel(
        lags=best_params['input_chunk_length'],
        output_chunk_length=1,  # Predicting one day at a time
        lags_future_covariates=[0],  # Using the current value of future covariates for prediction
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        min_child_weight=best_params['min_child_weight'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        gamma=best_params['gamma'],
        random_state=42
    )

    # Fit the model on the entire scaled training set
    best_model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled)

    # Make predictions on the test set
    forecast_scaled = best_model.predict(n=len(series_test), future_covariates=future_covariates_test_scaled)

    # Inverse transform the forecast to the original scale
    forecast = scaler_series.inverse_transform(forecast_scaled)

    return forecast

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Scaling the data
scaler_series = Scaler()
scaler_covariates = Scaler()

# Fit the scaler on the full training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Define list of n_splits to try
n_splits_list = [5, 10, 15]

# Initialize a dictionary to store DataFrames for each n_splits value
results_dict = {}

# Loop over different n_splits values
min_train_size = 200  # Define a minimum training size to ensure seasonality capture or other patterns
for n_splits in n_splits_list:
    print(f"Processing custom cross-validation with {n_splits} splits...")

    # Create custom cross-validation generator
    tscv_generator = custom_time_series_cv(series_train_scaled.pd_dataframe(), n_splits=n_splits, min_train_size=min_train_size)
    
    # Train and evaluate model using cross-validation with pruning
    study = train_and_evaluate(tscv_generator, series_train_scaled, future_covariates_train_scaled, scaler_series, n_splits)

    # Print the best hyperparameters
    print(f'Best hyperparameters for {n_splits} splits: ', study.best_params)

    # Store the best hyperparameters for each split setting
    best_params = study.best_params

    # --- Slicing the Test Covariates According to Best `input_chunk_length` ---
    input_chunk_length = best_params['input_chunk_length']
    required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=input_chunk_length)
    required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test) - 1)
    future_covariates_test = future_covariates_full.slice(required_start_date, required_end_date)
    future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

    # Make final predictions on the test set using the best hyperparameters
    forecast = final_model_evaluation(best_params, series_train_scaled, future_covariates_train_scaled, series_test, future_covariates_test_scaled, scaler_series)

    # Store the results
    results_dict[f'n_splits_{n_splits}'] = pd.DataFrame({
        'mse': [mse(series_test, forecast)],
        'mae': [mae(series_test, forecast)],
        'mape': [mape(series_test, forecast) if (series_test.values() > 0).all() else np.nan],
        'rmse': [rmse(series_test, forecast)]
    })

# Find the best n_splits based on the lowest mse
comparison_df = pd.concat(
    [df.assign(n_splits=int(n_splits.split('_')[-1])) for n_splits, df in results_dict.items()]
)

best_n_splits = comparison_df['n_splits'].iloc[comparison_df['mse'].idxmin()]

# Corrected: Format the key properly to match the results_dict keys
best_n_splits_key = f'n_splits_{best_n_splits}'  # No need to convert to int, as it's already the correct format

# Extract the best hyperparameters for the best n_splits
best_results_df = results_dict[best_n_splits_key]
best_trial = best_results_df.loc[best_results_df['mse'].idxmin()]

print("\nBest hyperparameter combination and error metrics for the best TimeSeriesSplit configuration:")
print(f"Best n_splits: {best_n_splits_key}")
print(f"Best hyperparameters: {study.best_params}")
print(f"MSE: {best_trial['mse']}")
print(f"MAE: {best_trial['mae']}")
print(f"MAPE: {best_trial['mape']}")
print(f"RMSE: {best_trial['rmse']}")

# Evaluate error metrics on the full test data using the best hyperparameters
mse_test = mse(series_test, forecast)
mae_test = mae(series_test, forecast)
mape_test = mape(series_test, forecast) if (series_test.values() > 0).all() else np.nan
rmse_test = rmse(series_test, forecast)

print("\nError metrics for the test data using the best hyperparameters:")
print(f"MSE on Test Set: {mse_test}")
print(f"MAE on Test Set: {mae_test}")
print(f"MAPE on Test Set: {mape_test}")
print(f"RMSE on Test Set: {rmse_test}")

# Plot the results using Plotly for the best hyperparameters
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='XGBoost Forecast on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title=f'XGBoost Model - Test Performance (Best n_splits={best_n_splits_key})',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly'
)

# Show the plot
fig.show()


Processing custom cross-validation with 5 splits...


/var/folders/8w/b_0gc01d70g6h4k62sb7ytyw0000gn/T/ipykernel_25368/841950022.py:120: ExperimentalWarning:

PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.



TypeError: PatientPruner.__init__() missing 1 required positional argument: 'patience'

In [78]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import XGBModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mse, mae
import plotly.graph_objs as go

# Convert future covariates to TimeSeries objects (if you have future covariates, otherwise omit this)
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Scaling the data
scaler_series = Scaler()
scaler_covariates = Scaler()

# Fit the scaler on the full training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# --- XGBoost Model ---
# Define the model
xgb_model = XGBModel(
    lags=120,  # Number of past lags to use for the model
    output_chunk_length=1,  # Predict one step ahead
    lags_future_covariates=[0],  # You need to specify the lags for future covariates
    max_depth=6,  # Example of hyperparameter
    n_estimators=500,  # Number of trees in the model
    learning_rate=0.03,  # Learning rate
    random_state=42
)

# Train the model on the scaled training data
xgb_model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled)

# --- Testing the model ---
# Scale the test data
series_test_scaled = scaler_series.transform(series_test)
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

# Make predictions on the test set
forecast_scaled = xgb_model.predict(n=len(series_test), future_covariates=future_covariates_test_scaled)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# --- Evaluate the performance ---
mse_test = mse(series_test, forecast)
mae_test = mae(series_test, forecast)
rmse_test = rmse(series_test, forecast)

# Only calculate MAPE if actual values are positive
if (series_test.values() > 0).all():
    mape_test = mape(series_test, forecast)
else:
    mape_test = np.nan

print(f"MSE on Test Set: {mse_test}")
print(f"MAE on Test Set: {mae_test}")
print(f"MAPE on Test Set: {mape_test}")
print(f"RMSE on Test Set: {rmse_test}")

# --- Plot the results using Plotly ---
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='XGBoost Forecast on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='XGBoost Model - Test Performance',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   
        y=1,   
        xanchor='right',  
        yanchor='top',    
        bordercolor='black',  
        borderwidth=1        
    ),
    template='plotly_white'
)

# Show the plot
fig.show()


MSE on Test Set: 4220.7763671875
MAE on Test Set: 44.283077239990234
MAPE on Test Set: nan
RMSE on Test Set: 64.96749877929688
